<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re
from function_library import *
from function_library2 import *

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# namelist= ['Marks','Dailey','Desloge','Miller','Mustian','Sauls','Ziffer','Gillum','Maddox','Lindley','Dozier','Proctor','Richardson']
# for i in range(len(namelist)):
#     namelist[i] = namelist[i].lower()
# keep_idx=[]
# for i in range(len(from_list)):
#     for name in namelist:
#         if name in from_list[i].lower() or name in to_list[i].lower() or name in cc_list[i].lower():
#             keep_idx.append(i)
#             break

# len(keep_idx)

In [6]:
re_behalf = re.compile(r'(.*)[Oo]n [Bb]ehalf')
re_bracket = re.compile(r'(.*)[\[\<\(]mailto\:(.*)[\]\>\)]')
re_bracket2 = re.compile(r'(.*)[\[\<\(](.*)[\]\>\)]')

re_name1 = re.compile(r'.*?([A-Za-z]+\s?[A-Z]?\.?)\s([A-Za-z]+)')
re_name2 = re.compile(r'.*?([A-Za-z]+),\s([A-Za-z]+\s?[A-Z]?\.?)')
re_email = re.compile(r'.*?([a-zA-Z_]*\.?\w+@[a-zA-Z_0-9]*\.?[a-zA-Z_]*\.?[a-zA-Z]{2,3})')


In [7]:
# either the sender or the recipient
df = pd.read_csv('new_clean_output.csv',index_col = 0)
df = df.drop_duplicates(keep='first',subset=['Sent'])
df = df.reset_index(drop=True)

from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()

try:
    del df['title_length_c']
    del df['title_length_w']
    del df['body_length_w']
    del df['POS']
    del df['POS_dict']
except:
    pass


In [8]:
# test
sss = 'McGee, Michele'.lower()
sss1 = 'sarah_aviles@85cbakerycafe.com'
sss2 = 'Elias Mathes TDIA'
sss3 = 'Adam B Corey'
print(re_name2.findall(sss))

sss4 = 'Jim Murdaugh Ph. D. (murdaugj@tcc.fl.edu); Will Butler (w.f.butler@embarqm.com)'

for t in sss4.split(';'):
    print(t)
    if re_bracket2.match(t):
        print('match')

[('mcgee', 'michele')]
Jim Murdaugh Ph. D. (murdaugj@tcc.fl.edu)
match
 Will Butler (w.f.butler@embarqm.com)
match


In [9]:
people_list = []
named_email_list = []
name_list = []
# find unique complete person in From section

for f in from_list: 
    f = f[2:-2].lower()
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
    if re_bracket.match(f):
        tname, temail = re_bracket.findall(f)[0]
        tname=tname.lower().strip()
        temail=temail.lower()
        
#         print(tname, temail)
        if re_email.match(temail) and re_email.match(tname) == None:
            email = re_email.findall(temail)[0]
            if len(tname.split()) != 2:
                first_name = tname
                last_name = ' '
            else:
                if re_name1.match(tname):
                    name = re_name1.findall(tname)[0]
                    first_name = name[0]
                    last_name = name[1]
                elif re_name2.match(tname): 
                    name = re_name2.findall(tname)[0]
                    first_name = name[1]
                    last_name = name[0]
                else:
                    print('error: cannot find name in tname')
                    print('tname:',tname)
                    break
            if email not in named_email_list:
                person = (first_name, last_name, email)
                people_list.append(person)
                named_email_list.append(email)
                name_list.append(first_name + ' ' + last_name)
            else:
#                 print(person)
                pass

In [10]:
len(people_list)

407

In [11]:
# find unique complete person in TO section
for ts in to_list: 
    ts = ts[2:-2].lower()
    ts = ts.split(';')
    for t in ts:
        t = t.strip("'")

        if re_bracket.match(t) or re_bracket2.match(t):
            if re_bracket.match(t):
                tname, temail = re_bracket.findall(t)[0]
            else:
                tname, temail = re_bracket2.findall(t)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        break
                        print('error: cannot find name in tname')
                        print('tname:',tname)
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                    
#                     print(person)
#                     print(tname)
#                     print(t)
                
                else:
#                     print(person)
                    pass

In [12]:
len(people_list)

1724

In [13]:
# find unique complete person in CC section
for ccs in cc_list: 
    ccs = ccs[2:-2].lower()
    ccs = ccs.split(';')
    for cc in ccs:
        cc = cc.strip("'")

        if re_bracket.match(cc) or re_bracket2.match(cc):
            if re_bracket.match(cc):
                tname, temail = re_bracket.findall(cc)[0]
            else:
                tname, temail = re_bracket2.findall(cc)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        break
                        print('error: cannot find name in tname')
                        print('tname:',tname)
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                    
#                     print(person)
#                     print(tname)
#                     print(cc)
                
                else:
#                     print(person)
                    pass

In [14]:
len(people_list)

1941

In [36]:
from_list = [ "['1 Joey asdf']", "['101 restarant']"]

In [38]:
new_from_list = []

unknown_idx = 0
# replace the From section with unique people information

for f in from_list: 
    email_exist_flag = 0
    f = f[2:-2].lower().strip("'")
    email = ''
    first_name = ''
    last_name = ''
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
        
    if re_email.match(f):
        email = re_email.findall(f)[0]
        email = email.lower()
        email_exist_flag = 1
    if len(f.split()) != 2:
        first_name = f
        last_name = ' '
    else:
        if re_name1.match(f):
            name = re_name1.findall(f)[0]
            first_name = name[0]
            last_name = name[1]
        elif re_name2.match(f): 
            name = re_name2.findall(f)[0]
            first_name = name[1]
            last_name = name[0]
        elif email_exist_flag == 1:
            pass
        else:
            print('error: cannot find name and email in f')
            print('f:', f)
            first_name = 'fake'
            last_name = f
            
    name = first_name + ' ' + last_name

    
    if email in named_email_list:
        idx = named_email_list.index(email)
        new_from_list.append(people_list[idx])
    elif name in name_list:
        idx = name_list.index(name)
        new_from_list.append(people_list[idx])
    elif email_exist_flag == 1:
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
        new_from_list.append(person)
        unknown_idx = unknown_idx + 1
    else:
        person = (first_name, last_name, first_name + '_' + last_name)
        new_from_list.append(person)

error: cannot find name and email in f
f: 101 restarant


In [41]:
new_from_list

[('1 joey asdf', ' ', '1 joey asdf_ '),
 ('fake', '101 restarant', 'fake_101 restarant')]

In [39]:
len(from_list)

2

In [17]:
len(new_from_list)

10010

In [281]:
new_to_lists = []

# replace the To section with unique people information

for ts in to_list: 
    ts = ts[2:-2].lower()
    ts = ts.split(';')
    new_to_list = []
    for t in ts:
        t = t.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(t):
            t = re_behalf.findall(t)[0]

        if re_email.match(t):
            email = re_email.findall(t)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(t.split()) != 2:
            first_name = t
            last_name = ' '
        else:
            if re_name1.match(t):
                name = re_name1.findall(t)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(t): 
                name = re_name2.findall(t)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = t
                print('error: cannot find name and email in t, make fake name')
                print('t:', t)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_to_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_to_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_to_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_to_list.append(person)
    new_to_lists.append(new_to_list)

In [282]:
len(to_list)

10010

In [283]:
len(new_to_lists)

10010

In [262]:
new_to_lists[:10]

[[('Bill', 'Behenna', 'Bill_Behenna'),
  ('Cathy', 'Branch', 'Cathy_Branch'),
  ('Paula', 'Burn', 'Paula_Burn'),
  ('Stacey', 'Campbell', 'Stacey_Campbell'),
  ('Tom', 'Coe', 'Tom_Coe'),
  ('Cindy', 'Cook', 'Cindy_Cook'),
  ('Jim', 'English', 'Jim_English'),
  (' Favors Thompson, Anita', ' ', ' Favors Thompson, Anita_ '),
  ('Carol', 'Gerrell', 'Carol_Gerrell'),
  ('gil', 'ziffer', 'gil@ziffberry.com'),
  ('Fred', 'Goodrow', 'Fred_Goodrow'),
  (' Griffin,Phyllis', ' ', ' Griffin,Phyllis_ '),
  ('Jacqueline', 'Hightower', 'Jacqueline_Hightower'),
  (' Holly', ' ', ' Holly_ '),
  ('M', 'Ingram', 'M_Ingram'),
  (' BethKirkland', ' ', ' BethKirkland_ '),
  ('Raoul', 'Lavin', 'Raoul_Lavin'),
  ('John', 'Marks', 'John_Marks'),
  ('Rick', 'McCraw', 'Rick_McCraw'),
  ('Rick', 'Minor', 'Rick_Minor'),
  ('Ken', 'Morris', 'Ken_Morris'),
  ('Kara', 'Palmer', 'Kara_Palmer'),
  ('Michael', 'Parker', 'Michael_Parker'),
  ('Lewis', 'Shelley', 'Lewis_Shelley'),
  ('Sullivan, Sherri L.', ' ', 'Sullivan,

In [284]:
new_cc_lists = []

# replace the cc section with unique people information

for ccs in cc_list: 
    ccs = ccs[2:-2].lower()
    ccs = ccs.split(';')
    new_cc_list = []
    if len(ccs) == 1 and ccs[0].strip() == '':
        new_cc_lists.append(new_cc_list)
        continue
    for cc in ccs:
        cc = cc.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(cc):
            cc = re_behalf.findall(cc)[0]

        if re_email.match(cc):
            email = re_email.findall(cc)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(cc.split()) != 2:
            first_name = cc
            last_name = ' '
        else:
            if re_name1.match(cc):
                name = re_name1.findall(cc)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(cc): 
                name = re_name2.findall(cc)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = cc
                print('error: cannot find name and email in cc, make fake name')
                print('cc:', cc)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_cc_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_cc_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_cc_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_cc_list.append(person)
    new_cc_lists.append(new_cc_list)

In [285]:
len(cc_list)

10010

In [286]:
len(new_cc_lists)

10010

In [287]:
new_cc_lists[:10]

[[],
 [('f19308', 'l19308', 'brook.pace@deo.myflorida.com'),
  ('kim', 'rivers', 'kim@inkbridge.com'),
  ('jt', 'burnette', 'jt@inkbridge.com'),
  ('todd', 'kocourek', 'todd_kocourek'),
  ('james', 'hosman', 'james_hosman'),
  (" 'parker,michael", ' ', " 'parker,michael_ ")],
 [],
 [],
 [],
 [],
 [('laura', 'youmans', 'laura_youmans')],
 [('laura', 'youmans', 'laura_youmans')],
 [('dee', 'crumpler', 'dee.crumpler@talgov.com')],
 [('dee', 'crumpler', 'dee.crumpler@talgov.com')]]

In [288]:
df_new = df.copy()
df_new['From'] = new_from_list
df_new['To'] = new_to_lists
df_new['CC'] = new_cc_lists
df_new


,From,Sent,To,CC,Subject,Attachments,Body
0,"(deborah, heiter, deborah.heiter@talgov.com)",2012-01-20 16:41:00,"[(bill, behenna, bill_behenna), (cathy, branch...",[],['Economic Development Target Issue Meeting'],[],"['', '', '', 'Deborah L. Heiter', 'Administrat..."
1,"(beth, kirkland, bkirkland@taledc.com)",2012-01-23 09:32:00,"[(f320, l320, llaubscher@eflorida.com)]","[(f19308, l19308, brook.pace@deo.myflorida.com...",['City of Tallahassee meeting regarding SBCI'],[],"['', 'Louis,', '', 'Thank you for meeting with..."
2,"(wiebler, brian t., , brian.wiebler@talgov.com)",2012-02-27 10:04:00,"[(, , _ )]",[],['Rural Density'],[],"['', '', '']"
3,"(wiebler, brian t., , brian.wiebler@talgov.com)",2012-03-01 15:51:00,"[(wiebler, brian t., , brian.wiebler@talgov.c...",[],['Update on Rural Density Comprehensive Plan T...,[],"['', 'Thank you for your interest in the upcom..."
4,"(wiebler, brian t., , brian.wiebler@talgov.com)",2012-03-27 15:59:00,"[(wiebler, brian t., , brian.wiebler@talgov.c...",[],['Rural Density Comprehensive Plan Text Amendm...,[],"['', 'This is a reminder of the open house eve..."
5,"(gary, yordon, gary@govinc.net)",2012-04-02 11:18:00,"[(wiebler, brian t., , brian.wiebler@talgov.c...",[],['Rockaway'],[],"['', 'Brian,', '', 'I just talked with Luis Se..."
6,"(wiebler, brian t., , brian.wiebler@talgov.com)",2012-04-02 15:15:00,"[(gary, yordon, gary@govinc.net)]","[(laura, youmans, laura_youmans)]",['RE: Rockaway'],[],"['', '', 'Hi Gary,', 'Laura Youmans from the C..."
7,"(gary, yordon, gary@govinc.net)",2012-04-02 16:59:00,"[(wiebler, brian t., , brian.wiebler@talgov.c...","[(laura, youmans, laura_youmans)]",['Re: Rockaway'],[],"['', '', 'Thanks Brian. I have left a message ..."
8,"(sheila, costigan, costigans@leonschools.net)",2012-04-02 21:39:00,"[(thomas, harrison, thomas_harrison), (april, ...","[(dee, crumpler, dee.crumpler@talgov.com)]","['FLCS BOD ANNUAL RETREAT- please open, note a...",[],"['', '', '', 'Everyone, please respond to this..."
9,"(diana, oropallo, doropallo@comcast.net)",2012-04-02 22:07:00,"[(sheila, costigan, costigans@leonschools.net)...","[(dee, crumpler, dee.crumpler@talgov.com)]","['RE: FLCS BOD ANNUAL RETREAT- please open, no...",[],"['', '', '', 'Just to piggy back on Sheilas em..."


In [289]:
df

,From,Sent,To,CC,Subject,Attachments,Body
0,"['Heiter, Deborah [mailto:Deborah.Heiter@talgo...",2012-01-20 16:41:00,"[""Behenna, Bill; Branch, Cathy; Burn, Paula; C...",[],['Economic Development Target Issue Meeting'],[],"['', '', '', 'Deborah L. Heiter', 'Administrat..."
1,['Beth Kirkland [mailto:bkirkland@taledc.com]'],2012-01-23 09:32:00,"[""'llaubscher@eflorida.com'""]","[""brook.pace@deo.myflorida.com; Kim Rivers(kim...",['City of Tallahassee meeting regarding SBCI'],[],"['', 'Louis,', '', 'Thank you for meeting with..."
2,"['Wiebler, Brian T.']",2012-02-27 10:04:00,[],[],['Rural Density'],[],"['', '', '']"
3,"['Wiebler, Brian T.']",2012-03-01 15:51:00,"['Wiebler, Brian T.']",[],['Update on Rural Density Comprehensive Plan T...,[],"['', 'Thank you for your interest in the upcom..."
4,"['Wiebler, Brian T.']",2012-03-27 15:59:00,"['Wiebler, Brian T.']",[],['Rural Density Comprehensive Plan Text Amendm...,[],"['', 'This is a reminder of the open house eve..."
5,['Gary Yordon [mailto:gary@govinc.net]'],2012-04-02 11:18:00,"['Wiebler, Brian T.']",[],['Rockaway'],[],"['', 'Brian,', '', 'I just talked with Luis Se..."
6,"['Wiebler, Brian T.']",2012-04-02 15:15:00,"[""'Gary Yordon'""]","['Youmans, Laura']",['RE: Rockaway'],[],"['', '', 'Hi Gary,', 'Laura Youmans from the C..."
7,['Gary Yordon'],2012-04-02 16:59:00,"['Wiebler, Brian T.']","['Youmans, Laura']",['Re: Rockaway'],[],"['', '', 'Thanks Brian. I have left a message ..."
8,"['Costigan, Sheila [mailto:costigans@leonschoo...",2012-04-02 21:39:00,"[""Administration-Thomas Harrison; april salter...","[""'Crumpler, Dee'""]","['FLCS BOD ANNUAL RETREAT- please open, note a...",[],"['', '', '', 'Everyone, please respond to this..."
9,['Diana Oropallo'],2012-04-02 22:07:00,"[""'Costigan, Sheila'; 'Administration-ThomasHa...","[""'Crumpler, Dee'""]","['RE: FLCS BOD ANNUAL RETREAT- please open, no...",[],"['', '', '', 'Just to piggy back on Sheilas em..."


In [290]:
df_new.to_csv('newest_cleaned_output.csv')

In [291]:
df_org=pd.read_excel('emails-combine 06162020.xlsx',sheet_name=0,usecols=[1,2,3,15,19,23])
gorvenment_people_list = []
for i in range(len(df_org)):
    row = df_org.iloc[i]
    if row['Organization1'] is not np.nan:
        if 'Tallahassee' in row['Organization1'] or 'Leon County' in row['Organization1']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue
    if row['Organization 2'] is not np.nan:
        if 'Tallahassee' in row['Organization 2'] or 'Leon County' in row['Organization 2']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue
    if row['Organization 3'] is not np.nan:
        if 'Tallahassee' in row['Organization 3'] or 'Leon County' in row['Organization 3']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue

In [301]:
df_cra=pd.read_excel('emails-combine 06162020.xlsx',sheet_name=1,usecols=[1,2])
df_cra = df_cra.iloc[2:16]
df_cra.columns = ['first','last']
df_cra = df_cra.applymap(str.lower)
df_cra = df_cra.reset_index(drop=True)


In [302]:
df_cra

,first,last
0,john,marks
1,john,dailey
2,george,desloge
3,jessica,miller
4,mark,mustian
5,phillip,sauls
6,gil,ziffer
7,andrew,gillum
8,nick,maddox
9,scott,maddox


In [303]:
first = df_cra['first'].values.tolist()
last = df_cra['last'].values.tolist()

In [305]:
'carter' in first

False

In [310]:
df_new['Sent'] = datetime(df_new['Sent'].values.tolist())

# .dt.year

TypeError: an integer is required (got type list)